# HealthCare Dataset Explorer

## Imports

In [72]:
import pandas as pd
from IPython.display import display, Markdown
from decimal import Decimal

## Functions

In [73]:
def dmd(text):
    display(Markdown(text))


def remove_na(df,column):
    mask = df[column].isna()
    rows_to_log = df[mask]
    df.drop(df[mask].index, inplace=True)
    dmd(rows_to_log.to_string())

## Loading data

In [74]:
df_hc = pd.read_csv("../data/healthcare_dataset.csv", dtype = {
    'Name' : str,
    'Age' : int,
    'Gender' : str,
    'Blood Type': str,
    'Medical Condition' : str,
    'Date of Admission' : object,
    'Doctor' : str,
    'Hospital' : str,
    'Insurance Provider' : str,
    'Billing Amount' : float,
    'Room Number' : int,
    'Admission Type' : str,
    'Discharge Date' : object,
    'Medication' : str,
    'Test Results': str
    })
df_hc['Date of Admission'] = pd.to_datetime(df_hc['Date of Admission']).dt.date
df_hc['Discharge Date'] = pd.to_datetime(df_hc['Discharge Date']).dt.date
df_hc['Billing Amount'] = df_hc['Billing Amount'].round(2)

In [64]:
df_hc.describe(include='all')

Column Name,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
count,55500,55500.000000,55500,55500,55500,55500,55500,55500,55500,55500.000000,55500.000000,55500,55500,55500,55500
unique,40235,NaN,2,8,6,1827,40341,39876,5,NaN,NaN,3,1856,5,3
top,Michael Williams,NaN,Male,A-,Arthritis,2024-03-16,Michael Smith,LLC Smith,Cigna,NaN,NaN,Elective,2020-03-15,Lipitor,Abnormal
freq,24,NaN,27774,6969,9308,50,27,44,11249,NaN,NaN,18655,53,11140,18627
mean,NaN,51.539459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25539.316071,301.134829,NaN,NaN,NaN,NaN
std,NaN,19.602454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14211.454434,115.243069,NaN,NaN,NaN,NaN
min,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2008.490000,101.000000,NaN,NaN,NaN,NaN
25%,NaN,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13241.222500,202.000000,NaN,NaN,NaN,NaN
50%,NaN,52.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25538.065000,302.000000,NaN,NaN,NaN,NaN
75%,NaN,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37820.510000,401.000000,NaN,NaN,NaN,NaN


In [71]:
df_hc.columns

Index(['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition',
       'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider',
       'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date',
       'Medication', 'Test Results'],
      dtype='object', name='Column Name')

## Patient data cleaning

In [70]:
df_hc['Name'] = df_hc['Name'].str.title()
remove_na(df_hc,'Name')

Empty DataFrame
Columns: [Name, Age, Gender, Blood Type, Medical Condition, Date of Admission, Doctor, Hospital, Insurance Provider, Billing Amount, Room Number, Admission Type, Discharge Date, Medication, Test Results]
Index: []

In [56]:
df_hc['Doctor'] = df_hc['Doctor'].str.title()

In [36]:
df_hc['Blood Type'] = df_hc['Blood Type'].str.upper()
df_hc['Blood Type'].value_counts()

Blood Type
AB+    6291
B+     6278
A-     6261
AB-    6259
A+     6246
O+     6235
B-     6230
O-     6200
Name: count, dtype: int64

In [37]:
df_hc['Age'] = df_hc['Age'].astype(int)
df_hc['Age'].value_counts().sort_index()

Age
15     14
16     28
17     38
18    690
19    738
     ... 
83    705
84    690
85    672
86     36
87      8
Name: count, Length: 73, dtype: int64

### Search for medical data inconsistency

In [38]:
df_hc['Gender'] = df_hc['Gender'].str.title()
df_hc['Gender'].value_counts()

Gender
Female    25011
Male      24989
Name: count, dtype: int64

Blood Type
A+     6956
A-     6969
AB+    6947
AB-    6945
B+     6945
B-     6944
O+     6917
O-     6877
Name: count, dtype: int64

In [ ]:
df_hc['Blood Type'] = df_hc['Blood Type'].str.upper()

In [17]:
df_hc.groupby('Doctor')[['Hospital']].value_counts()

Doctor           Hospital                    
Aaron Acevedo    Cervantes, and Mckenzie Byrd    1
Aaron Adams      LLC Boyle                       1
Aaron Aguilar    Hess White and Drake,           1
Aaron Alexander  Richardson-Juarez               1
Aaron Anderson   Miller-Stanton                  1
                                                ..
Zoe Khan         Hunter PLC                      1
Zoe Knight       and Sons Walters                1
Zoe Nichols      Harris-Lee                      1
Zoe Roberts      and Williams Garza, Herman      1
Zoe Wallace      Knight Wilkerson, and Carey     1
Name: count, Length: 50000, dtype: int64

In [6]:
display(df_hc.shape)
df_hc.describe(include='all')

(55500, 15)

Column Name,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
count,55500,55500.000000,55500,55500,55500,55500,55500,55500,55500,55500.000000,55500.000000,55500,55500,55500,55500
unique,40235,NaN,2,8,6,1827,40341,39876,5,NaN,NaN,3,1856,5,3
top,Michael Williams,NaN,Male,A-,Arthritis,2024-03-16,Michael Smith,LLC Smith,Cigna,NaN,NaN,Elective,2020-03-15,Lipitor,Abnormal
freq,24,NaN,27774,6969,9308,50,27,44,11249,NaN,NaN,18655,53,11140,18627
mean,NaN,51.539459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25539.316097,301.134829,NaN,NaN,NaN,NaN
std,NaN,19.602454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14211.454431,115.243069,NaN,NaN,NaN,NaN
min,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2008.492140,101.000000,NaN,NaN,NaN,NaN
25%,NaN,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13241.224652,202.000000,NaN,NaN,NaN,NaN
50%,NaN,52.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25538.069376,302.000000,NaN,NaN,NaN,NaN
75%,NaN,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37820.508436,401.000000,NaN,NaN,NaN,NaN


## Duplicated process

### Remove all but 'Age' columns duplicated

In [75]:
subset_cols = (df_hc.columns.to_list())
subset_cols.remove('Age')

In [76]:
df_hc = df_hc.groupby(subset_cols)['Age'].mean().reset_index()
df_hc['Age'] = df_hc['Age'].astype(int)
df_hc

,Name,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Age
0,AARON DuncAn,Male,AB-,Obesity,2019-07-24,Ryan Perry,Welch-Yang,UnitedHealthcare,39906.15,279,Urgent,2019-08-04,Paracetamol,Abnormal,22
1,AARON HicKS,Female,A+,Arthritis,2022-03-02,Michael Butler,"Rasmussen Patrick and Newman,",Cigna,10584.19,187,Elective,2022-03-15,Lipitor,Inconclusive,76
2,AARON bAldWIN Jr.,Male,O-,Hypertension,2020-10-10,Amy Farley,"Flores Friedman and White,",Medicare,29740.96,104,Urgent,2020-11-05,Paracetamol,Abnormal,20
3,AARON hAWkIns,Female,B-,Diabetes,2019-10-17,Kimberly York,"Harris, Hernandez and Vazquez",Aetna,21535.55,206,Urgent,2019-10-26,Penicillin,Abnormal,69
4,AAROn HaRt,Male,B-,Cancer,2021-01-13,Sharon Morrison,"Fox Guzman James, and",Aetna,13895.55,260,Emergency,2021-01-20,Paracetamol,Abnormal,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,zachaRy huLL,Female,B-,Asthma,2022-12-08,Sarah Martinez,"Miller Santana and Kennedy,",Medicare,41876.90,128,Elective,2022-12-15,Lipitor,Normal,53
49996,zachaRy oDOM,Female,O+,Diabetes,2021-07-08,Sherry Durham,Group Turner,Blue Cross,48301.35,326,Urgent,2021-07-10,Paracetamol,Abnormal,52
49997,zachaRy raMirEZ,Male,AB+,Asthma,2019-06-30,Heather Chen,"and Waters, Williams Daugherty",Aetna,29508.12,205,Emergency,2019-07-20,Aspirin,Inconclusive,58
49998,zacharY BauTista,Female,AB+,Cancer,2020-08-21,Scott Bell,Rodgers Inc,Medicare,9988.20,336,Urgent,2020-08-25,Aspirin,Inconclusive,44


In [ ]:
df_hc[df_hc.duplicated(subset=['Name', 'Date of Admission'], keep= False)].sort_values('Name')

,Name,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Age


In [57]:
df_hc.index

RangeIndex(start=0, stop=55500, step=1)